<a href="https://colab.research.google.com/github/GeniusJr001/deeplearning/blob/main/Copy_of_fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-11-28 10:55:05--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   155MB/s    in 0.2s    

2023-11-28 10:55:05 (155 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Mutate count columns
df_ratings['count_users'] = df_ratings.groupby(['user'], group_keys=False).transform('count')['rating']
df_ratings['count_books'] = df_ratings.groupby(['isbn'], group_keys=False).transform('count')['rating']

# Clean data
df_ratings = df_ratings.loc[(df_ratings.count_users >= 200) & (df_ratings.count_books >= 100)]
df_ratings = df_ratings[['user','isbn','rating']]

# Merge dataframes
df = pd.merge(df_ratings, df_books, on="isbn")

# Remove duplicates (Same user several reviews over same book)
df = df.drop_duplicates(subset=['user','title'])

# Transpose dataframe to have tittle vs users with values in each cell. Introduce 0's for NAs.
dfknn = df.pivot(index='title', columns='user', values='rating').fillna(0)

dfknn.shape

# Get list of titles (Books)
titles = list(dfknn.index.values)

# Functions to transform title to index and get row values for the title
def title_2_index(data, title, titles=titles):
  ind = titles.index(title)
  return ind, data.values[ind,:]

def index_2_title(ind, titles=titles):
  return titles[ind]

# Examples modify title to index
x, y = title_2_index(dfknn, '1984')
print('Index:', x)
print('First 10 values in the row:', y[1:10])
print('Book name from index:', index_2_title(x))

# Model (Euclidean distance - p=2)
neigh = NearestNeighbors(metric="cosine",algorithm="brute")
neigh.fit(dfknn.values)


Index: 0
First 10 values in the row: [0. 0. 0. 0. 0. 0. 0. 0. 0.]
Book name from index: 1984


NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  ind, val = title_2_index(dfknn, book)
  pred = neigh.kneighbors(np.reshape(val,[1,-1]),len(titles),return_distance=True)
  # I give all the list but extract the last 5 (Highest probability)
  prob_val = pred[0][0][1:5+1]
  prob_ind = pred[1][0][1:5+1]
  pred = []
  for num in range(len(prob_val)):
    pred.append([index_2_title(prob_ind[num]), prob_val[num]])
  # Create output
  pred = pred[::-1]
  recommended_books = [book, pred]

  return recommended_books

In [ ]:
print(get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))"))


['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Catch 22', 0.7939835], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Interview with the Vampire', 0.73450685], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]
